In [1]:
import torch

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

### Load Data

In [2]:
# In this code example we're gonna use different example, using data from Fish Market dataset
df = pd.read_csv("dataset.csv")
df.head()

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340


In [ ]:
sns.pairplot(df, hue='Species')
plt.show()

In [ ]:
df.head()

### Sample Data

In [ ]:
# We're gonna use data from one species (Bream) and sample several data points
# The independent variable (x) will be Height
# The dependent variable (y) will be Width
df_sample = df[(df['Species'] == 'Bream') & (df.index.isin([0, 11, 16, 27, 32]))][['Species', 'Height', 'Width']]
df_sample.head()

In [ ]:
X = df_sample['Height']
y = df_sample['Width']

plt.xlabel("Height")
plt.ylabel("Width")
plt.scatter(X, y)
plt.show()

### Define Function & Initial Param

In [ ]:
# We want to fit a line to the above data
# A line has an equation of y = mx + c
# We define a function to calculate y_predicted given parameters (m, c) and data (x)
def linreg_forward(m, x, c):
    return m * x + c

In [ ]:
# We define a function to calculate the cost function (error) of the predicted results
def mse(y_predicted, y_actual):
    return (y_predicted - y_actual)**2

In [ ]:
# We can start with an arbitrary value of parameters
np.random.seed(5)
m = torch.tensor(np.random.randn(1), requires_grad=True)
c = torch.tensor(np.random.randn(1), requires_grad=True)

m, c

### Start SGD

#### Step 1: Start with one data sample

In [ ]:
df_sample_shuffled = df_sample.sample(frac=1, random_state=5) # Shuffle
xi = torch.tensor(df_sample_shuffled['Height'].values[0])
yi = torch.tensor(df_sample_shuffled['Width'].values[0])

xi, yi

#### Step 2: Forward pass

In [ ]:
y_predicted = linreg_forward(m, xi, c)
y_predicted

#### Step 3: Calculate the cost function (error)

In [ ]:
error = mse(y_predicted, yi)
error

#### Step 4: Backprop, calculate the derivatives of cost function w.r.t. to parameters (m, c) 

In [ ]:
# This is calculated automatically using pytorch
error.backward()

In [ ]:
# The gradient of parameter m, c in current training sample
m.grad, c.grad

In [ ]:
# This is calculated manually by calculating derivative w.r.t. to each parameters

# Given the cost function MSE = (y_predicted - y_actual)^2
# First, we find the derivative of cost function w.r.t. to y_predicted, which is = 2(y_predicted - y_actual)

# We now calculate the derivative of the cost function w.r.t. to model parameter m
# The derivative will be derivative of cost function w.r.t. to y_predicted * derivative of y_predicted w.r.t. model parameter m
# We got the derivatives of y_predicted w.r.t. model parameter m is = x
# So, the derivative of the cost function w.r.t. to model parameter m will be = 2(y_predicted - y_actual) * x = 2x(y_predicted - y_actual)

# We now calculate the derivative of the cost function w.r.t. to model parameter c
# The derivative will be derivative of cost function w.r.t. to y_predicted * derivative of y_predicted w.r.t. model parameter c
# We got the derivatives of y_predicted w.r.t. model parameter c is = 1
# So, the derivative of the cost function w.r.t. to model parameter c will be = 2(y_predicted - y_actual) * 1 = 2(y_predicted - y_actual)

grad_m = 2 * xi * (y_predicted.item() - yi)
grad_c = 2 * (y_predicted.item() - yi)

grad_m, grad_c

#### Step 5: Update weight parameters

In [ ]:
# Here is the old m and c value
m, c

In [ ]:
plt.xlabel("Height")
plt.ylabel("Width")

plt.scatter(X, y)
plt.plot(X, m.item() * X + c.item())

plt.title("Line fit with m=%s, c=%s" % (round(m.item(), 3), round(c.item(), 3)))
plt.show()

In [ ]:
# Set learning rate to update the weight parameter
lr = 0.001
m = m - (lr * grad_m)
c = c - (lr * grad_c)

In [ ]:
# New model parameters
m, c

In [ ]:
plt.xlabel("Height")
plt.ylabel("Width")

plt.scatter(X, y)
plt.plot(X, m.item() * X + c.item())

plt.title("Line fit with m=%s, c=%s" % (round(m.item(), 3), round(c.item(), 3)))
plt.show()